In [1]:
!pip install topmost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 50.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 83.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [2]:
import numpy as np
from tqdm import tqdm
import scipy.sparse
from topmost.preprocessing import Preprocessing
import topmost.preprocessing.preprocessing as prepro
import importlib
from topmost.utils.logger import Logger
import fasttext
import fasttext.util
import nltk
from nltk.corpus import stopwords

## FastText

In [3]:
logger = Logger("WARNING")
logger = Logger("DEBUG")
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))
fasttext.util.download_model('es', if_exists='ignore')  # Español

# print(stopwords_es)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'cc.es.300.bin'

In [4]:
def make_word_embeddings_es(vocab):
    # Cargar el modelo binario preentrenado
    fasttext_model = fasttext.load_model("cc.es.300.bin")

    # glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
    word_embeddings = np.zeros((len(vocab), fasttext_model.get_dimension()))

    num_found = 0

    for i, word in enumerate(tqdm(vocab, desc="loading SPANISH 🍀 word embeddings")):
        word_embeddings[i] = fasttext_model.get_word_vector(word)
        num_found += 1

    logger.info(f'number of found embeddings: {num_found}/{len(vocab)}')

    return scipy.sparse.csr_matrix(word_embeddings)

prepro.make_word_embeddings = make_word_embeddings_es

## GloVe

In [ ]:
!mkdir -p /kaggle/working/modelos/gloVe  # Crea la carpeta si no existe

In [ ]:
# Modelo binario
!wget -c "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.bin" -P /kaggle/working/modelos/gloVe

In [3]:
# Modelo KeyVectprs
!wget -c "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.bin" -P !mkdir -p /kaggle/working/modelos/gloVe

URL transformed to HTTPS due to an HSTS policy
--2025-02-20 18:27:57--  https://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.bin
Resolving dcc.uchile.cl (dcc.uchile.cl)... 192.80.24.11
Connecting to dcc.uchile.cl (dcc.uchile.cl)|192.80.24.11|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://users.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.bin [following]
--2025-02-20 18:27:58--  https://users.dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.bin
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from gensim.models import Word2Vec
from gensim.models.wrappers import GloVe
word2vec_model = Word2Vec.load("complete.model")

In [ ]:
def make_word_embeddings_es(vocab):
    # Cargar el modelo binario preentrenado
    word2vec_model = Word2Vec.load("complete.model")

    # glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
    word_embeddings = np.zeros((len(vocab), fasttext_model.get_dimension()))

    num_found = 0

    for i, word in enumerate(tqdm(vocab, desc="loading SPANISH 🍀 word embeddings")):
        word_embeddings[i] = fasttext_model.get_word_vector(word)
        num_found += 1

    logger.info(f'number of found embeddings: {num_found}/{len(vocab)}')

    return scipy.sparse.csr_matrix(word_embeddings)

prepro.make_word_embeddings = make_word_embeddings_es

## Word@Vec

In [ ]:
!mkdir -p /kaggle/working/modelos/word2vec
!wget -c "http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz" -P /kaggle/working/modelos/gloVe
!wget -c "http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz" -P /kaggle/working/modelos/gloVe

In [ ]:
def make_word_embeddings_es(vocab):
    # Cargar el modelo binario preentrenado
    word2vec_model = Word2Vec.load("complete.model")

    # glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
    word_embeddings = np.zeros((len(vocab), fasttext_model.get_dimension()))

    num_found = 0

    for i, word in enumerate(tqdm(vocab, desc="loading SPANISH 🍀 word embeddings")):
        word_embeddings[i] = fasttext_model.get_word_vector(word)
        num_found += 1

    logger.info(f'number of found embeddings: {num_found}/{len(vocab)}')

    return scipy.sparse.csr_matrix(word_embeddings)

prepro.make_word_embeddings = make_word_embeddings_es

# ------------------------------------------------

In [5]:
jsonlist_path = '/kaggle/input/revista-ciencias-mdicas-de-la-habana-cmed'
output_path = './datasets/CMed'
    
# os.makedirs(jsonlist_path, exist_ok=True)
# os.makedirs(output_path, exist_ok=True)

preprocessing = Preprocessing(min_doc_count=10, stopwords=stopwords_es)

rst = preprocessing.preprocess_jsonlist(dataset_dir=jsonlist_path, label_name="label")

preprocessing.save(output_path, **rst)

2025-01-12 08:46:26,676 - TopMost - Found training documents 1208 testing documents 658
2025-01-12 08:46:26,679 - TopMost - label2id: {'carta-al-editor': 0, 'ciencias-basicas-biomedicas': 1, 'ciencias-clinicas-y-patologicas': 2, 'ciencias-de-la-educacion': 3, 'ciencias-epidemiologicas-y-salubristas': 4, 'ciencias-quirurgicas': 5, 'ciencias-sociales': 6, 'ciencias-tecnologicas': 7, 'editorial': 8, 'estudios-bibliometricos-y-cienciometricos': 9, 'historico': 10, 'nefrologia-al-dia': 11, 'reflexiones': 12, 'semblanza': 13}
parsing texts: 100%|██████████| 1208/1208 [00:03<00:00, 375.99it/s]
2025-01-12 08:46:45,998 - TopMost - Real vocab size: 19384
2025-01-12 08:46:46,018 - TopMost - Real training size: 1208 	 avg length: 1335.378
parsing texts: 100%|██████████| 658/658 [00:01<00:00, 377.12it/s]
2025-01-12 08:46:48,739 - TopMost - Real testing size: 658 	 avg length: 1347.125
loading SPANISH 🍀 word embeddings: 100%|██████████| 19384/19384 [00:00<00:00, 85854.12it/s]
2025-01-12 08:46:57,070

## Entidades nombradas

## Spacy

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion. Apple is looking at buying U.K. startup for $1 billion.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY
Apple ORG
U.K. GPE
$1 billion MONEY


In [5]:
from spacy import displacy

displacy.render(doc, style='ent', jupyter=True)

## Proceso completo de tokenizacion

In [41]:
import spacy
# nlp = spacy.load('/kaggle/working/es_core_news_lg-3.8.0-py3-none-any.whl')
nlp = spacy.load('es_core_news_lg')

text = ("""Introducción: Se ha notado el ingreso de neonatos con mastitis, cuya procedencia 
geográfica es diversa, pero parece recaer en determinadas áreas geográficas de la 
provincia La Habana. 
Objetivo: Determinar la incidencia y procedencia geográfica de la mastitis neonatal 
en la provincia La Habana. 
Material y métodos:Estudio descriptivo, retrospectivo y transversal de neonatos con 
mastitis, ingresados en los servicios abiertos de Neonatología de los hospitales 
pediátricos universitarios "Juan Manuel Márquez" y "William Soler", en la provincia de 
La Habana, durante el año 2013 y primer trimestre del 2014. La información se 
obtuvo de los expedientes clínicos de los pacientes y los registros estadísticos del 
departamento de Epidemiología Hospitalaria de la provincia La Habana y esta se 
procesó mediante estadística descriptiva. 
Resultados: La incidencia de mastitis neonatal en la provincia de La Habana para el 
período de estudio fue de 3.56 x 1,000 nacidos vivos. La proporción de casos con 
mastitis neonatal es mayor para los nacidos en el hospital "Luis Díaz Soto" y 
procedentes del municipio Habana del Este, con diferencias cuando se compara con 
cada uno de los otros municipios de la provincia La Habana. 
Conclusiones: La incidencia de la mastitis neonatal durante el período estudiado en 
la provincia La Habana indica ser más elevado que lo comúnmente reportado, con un 
predominio de casos provenientes del municipio Habana del Este y nacidos en el 
hospital "Luis Díaz Soto", lo cual sugiere que pudieran existir allí condiciones medioambientales o de otro tipo, que faciliten la afección.
Palabras clave: Mastitis, neonatal, Recién nacido, Infección, infecciones bacterianas, 
infecciones de tejidos blandos, absceso de la glándula mamaria, incidencia.
""")


tokens = nlp(text)

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
words_1 = set([token.text for token in tokens if not token.text in stopwords])
words_2 = set([token.lemma_ for token in tokens if not token.text in stopwords])

#print(words_1)
#print(words_2)
#print([x for x in words_1 if x not in words_2])
#print([x for x in words_2 if x not in words_1])


#print([(x.text, x.lemma_) for x in tokens if x.text not in stopwords])

#print(len(words_1), len(words_2))


# Seleccionado: token.lemma_ ✅

In [42]:
print(len(text))

1785


In [66]:
from nltk.corpus import stopwords
stopwords_ = stopwords.words('spanish')

text_split = text.split(' ')
print(len(text_split))

for x in stopwords_:
    try:
        text_split.remove(x)
    except:
        pass

print(len(text_split))

doc = nlp(' '.join(text_split))

from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

263
237


In [9]:
from nltk.corpus import stopwords
from spacy.lang.es.stop_words import STOP_WORDS

stopwords_1 = stopwords.words('spanish')

# stopwords_2 = spacy.lang.es.stop_words.STOP_WORDS

print([x for x in stopwords_1 if not x in STOP_WORDS])
print(len([x for x in stopwords_1 if not x in STOP_WORDS]))

print(len(stopwords_1 ))

print('-'*50)

print([x for x in STOP_WORDS if not x in stopwords_1])
print(len([x for x in STOP_WORDS if not x in stopwords_1]))
print(len(STOP_WORDS ))

print([x for x in stopwords_1 if x in STOP_WORDS])
print(len([x for x in stopwords_1 if x in STOP_WORDS]))

# Mejor resultado: nltk ✅ porque la de spacy tiene muchos verbos

['estás', 'estáis', 'esté', 'estés', 'estemos', 'estéis', 'estén', 'estaré', 'estarás', 'estaremos', 'estaréis', 'estarán', 'estaría', 'estarías', 'estaríamos', 'estaríais', 'estarían', 'estabas', 'estábamos', 'estabais', 'estuve', 'estuviste', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera', 'estuvieras', 'estuviéramos', 'estuvierais', 'estuvieran', 'estuviese', 'estuvieses', 'estuviésemos', 'estuvieseis', 'estuviesen', 'estando', 'estada', 'estadas', 'estad', 'has', 'habéis', 'hayas', 'hayamos', 'hayáis', 'hayan', 'habré', 'habrás', 'habremos', 'habréis', 'habrán', 'habría', 'habrías', 'habríamos', 'habríais', 'habrían', 'habías', 'habíamos', 'habíais', 'hube', 'hubiste', 'hubimos', 'hubisteis', 'hubieron', 'hubiera', 'hubieras', 'hubiéramos', 'hubierais', 'hubieran', 'hubiese', 'hubieses', 'hubiésemos', 'hubieseis', 'hubiesen', 'habiendo', 'habido', 'habida', 'habidos', 'habidas', 'seas', 'seamos', 'seáis', 'seré', 'serás', 'seremos', 'seréis', 'serías', 'seríamos', 'seríais',